# Airbnb Supply Analysis: European Cities
## Business Intelligence Engineer - End-to-End Project

This analysis explores Airbnb supply patterns across major European cities using exploratory data analysis and provides actionable business insights.

**Data Limitations:**
- Time-related factors (seasons, holidays, events) not considered
- Demand data from guest bookings not available
- Focus: Supply-side analysis only


## 1. Data Loading and ETL Pipeline


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set style for better visualizations
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Configuration
DATA_DIR = Path('data')
CITIES = ['amsterdam', 'athens', 'barcelona', 'berlin', 'budapest', 
          'lisbon', 'london', 'paris', 'rome', 'vienna']
PERIODS = ['weekdays', 'weekends']


In [ ]:
def load_airbnb_data(data_dir, cities, periods):
    """ETL Function: Load and combine all Airbnb datasets"""
    all_data = []
    
    for city in cities:
        for period in periods:
            file_path = data_dir / f"{city}_{period}.csv"
            if file_path.exists():
                df = pd.read_csv(file_path)
                df['city'] = city.capitalize()
                df['period'] = period
                all_data.append(df)
                print(f"Loaded {city}_{period}.csv: {len(df)} records")
    
    combined_df = pd.concat(all_data, ignore_index=True)
    print(f"\nTotal records loaded: {len(combined_df)}")
    return combined_df

# Load all data
df = load_airbnb_data(DATA_DIR, CITIES, PERIODS)


## 2. Data Quality Assessment & Cleaning


In [ ]:
print("=== Data Shape ===")
print(f"Rows: {df.shape[0]}, Columns: {df.shape[1]}")

print("\n=== Column Information ===")
print(df.info())

print("\n=== Missing Values ===")
missing = df.isnull().sum()
print(missing[missing > 0] if missing.sum() > 0 else "No missing values")

print("\n=== Sample Data ===")
df.head()


In [ ]:
# Data Cleaning
def clean_data(df):
    """Clean and preprocess the dataset"""
    df_clean = df.copy()
    
    # Remove unnamed index column if exists
    if 'Unnamed: 0' in df_clean.columns:
        df_clean = df_clean.drop('Unnamed: 0', axis=1)
    
    # Handle missing values
    numeric_cols = df_clean.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        if df_clean[col].isnull().sum() > 0:
            df_clean[col].fillna(df_clean[col].median(), inplace=True)
    
    # Ensure boolean columns are properly formatted
    bool_cols = ['room_shared', 'room_private', 'host_is_superhost', 'multi', 'biz']
    for col in bool_cols:
        if col in df_clean.columns:
            df_clean[col] = df_clean[col].astype(bool)
    
    # Create derived features
    df_clean['price_per_person'] = df_clean['realSum'] / df_clean['person_capacity']
    df_clean['location_score'] = (df_clean['attr_index_norm'] + df_clean['rest_index_norm']) / 2
    
    return df_clean

df = clean_data(df)
print(f"Cleaned dataset shape: {df.shape}")
print(f"\nColumns: {list(df.columns)}")


## 3. SQL-like Analysis (Pandas Operations)


In [ ]:
# SQL-like Analysis 1: Top N cities by average price
print("=== Top 5 Cities by Average Price ===")
city_avg_price = df.groupby('city')['realSum'].agg(['mean', 'median', 'count']).reset_index()
city_avg_price.columns = ['city', 'avg_price', 'median_price', 'listing_count']
city_avg_price = city_avg_price.sort_values('avg_price', ascending=False)
print(city_avg_price.head(5))


In [ ]:
# SQL-like Analysis 2: Room type distribution by city
print("=== Room Type Distribution by City ===")
room_type_city = pd.crosstab(df['city'], df['room_type'], margins=True)
print(room_type_city)


In [ ]:
# SQL-like Analysis 3: Superhost performance analysis
print("=== Superhost vs Regular Host Comparison ===")
superhost_analysis = df.groupby('host_is_superhost').agg({
    'realSum': ['mean', 'median'],
    'guest_satisfaction_overall': 'mean',
    'cleanliness_rating': 'mean'
}).round(2)
print(superhost_analysis)


In [ ]:
# SQL-like Analysis 4: Weekend vs Weekday pricing
print("=== Weekend vs Weekday Pricing Analysis ===")
period_pricing = df.groupby('period')['realSum'].agg(['mean', 'median', 'std']).reset_index()
period_pricing.columns = ['period', 'avg_price', 'median_price', 'std_price']
print(period_pricing)

# Calculate price premium
weekend_avg = period_pricing[period_pricing['period'] == 'weekends']['avg_price'].values[0]
weekday_avg = period_pricing[period_pricing['period'] == 'weekdays']['avg_price'].values[0]
premium = ((weekend_avg - weekday_avg) / weekday_avg) * 100
print(f"\nWeekend price premium: {premium:.2f}%")


In [ ]:
# SQL-like Analysis 5: Top 3 cities by listing count
print("=== Top 3 Cities by Listing Count ===")
top_cities = df.groupby('city').size().reset_index(name='listing_count')
top_cities = top_cities.sort_values('listing_count', ascending=False).head(3)
print(top_cities)


## 4. Data Visualization


In [ ]:
# Visualization 1: Price Distribution by City
plt.figure(figsize=(14, 8))
df_sorted = df.sort_values('realSum')
sns.boxplot(data=df_sorted, x='city', y='realSum', palette='Set2')
plt.title('Price Distribution by City', fontsize=16, fontweight='bold')
plt.xlabel('City', fontsize=12)
plt.ylabel('Price (realSum)', fontsize=12)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
# Visualization 2: Room Type Distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Room type count
room_counts = df['room_type'].value_counts()
axes[0].pie(room_counts.values, labels=room_counts.index, autopct='%1.1f%%', startangle=90)
axes[0].set_title('Room Type Distribution', fontsize=14, fontweight='bold')

# Room type vs price
sns.boxplot(data=df, x='room_type', y='realSum', ax=axes[1])
axes[1].set_title('Price by Room Type', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Room Type')
axes[1].set_ylabel('Price')

plt.tight_layout()
plt.show()


In [ ]:
# Visualization 3: City-wise Supply Analysis
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 3.1: Average price by city
city_price = df.groupby('city')['realSum'].mean().sort_values(ascending=False)
axes[0, 0].barh(city_price.index, city_price.values, color='steelblue')
axes[0, 0].set_title('Average Price by City', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Average Price')

# 3.2: Listing count by city
city_count = df.groupby('city').size().sort_values(ascending=False)
axes[0, 1].bar(city_count.index, city_count.values, color='coral')
axes[0, 1].set_title('Number of Listings by City', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('City')
axes[0, 1].set_ylabel('Count')
axes[0, 1].tick_params(axis='x', rotation=45)

# 3.3: Superhost percentage by city
superhost_pct = df.groupby('city')['host_is_superhost'].mean() * 100
axes[1, 0].bar(superhost_pct.index, superhost_pct.values, color='green')
axes[1, 0].set_title('Superhost Percentage by City', fontsize=12, fontweight='bold')
axes[1, 0].set_xlabel('City')
axes[1, 0].set_ylabel('Percentage (%)')
axes[1, 0].tick_params(axis='x', rotation=45)

# 3.4: Average satisfaction by city
satisfaction = df.groupby('city')['guest_satisfaction_overall'].mean().sort_values(ascending=False)
axes[1, 1].barh(satisfaction.index, satisfaction.values, color='purple')
axes[1, 1].set_title('Average Guest Satisfaction by City', fontsize=12, fontweight='bold')
axes[1, 1].set_xlabel('Average Satisfaction Score')

plt.tight_layout()
plt.show()


In [ ]:
# Visualization 4: Correlation Analysis
numeric_cols = ['realSum', 'person_capacity', 'bedrooms', 'cleanliness_rating', 
                'guest_satisfaction_overall', 'dist', 'metro_dist', 
                'attr_index_norm', 'rest_index_norm', 'price_per_person', 'location_score']

correlation_matrix = df[numeric_cols].corr()

plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Feature Correlation Matrix', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()


In [ ]:
# Visualization 5: Weekend vs Weekday Comparison
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Price comparison
sns.boxplot(data=df, x='period', y='realSum', ax=axes[0])
axes[0].set_title('Price: Weekday vs Weekend', fontsize=12, fontweight='bold')

# Satisfaction comparison
sns.boxplot(data=df, x='period', y='guest_satisfaction_overall', ax=axes[1])
axes[1].set_title('Satisfaction: Weekday vs Weekend', fontsize=12, fontweight='bold')

# Listing count
period_count = df['period'].value_counts()
axes[2].bar(period_count.index, period_count.values, color=['skyblue', 'lightcoral'])
axes[2].set_title('Listing Count by Period', fontsize=12, fontweight='bold')
axes[2].set_ylabel('Count')

plt.tight_layout()
plt.show()


## 5. Statistical Analysis


In [ ]:
from scipy import stats

# Statistical Test 1: Weekend vs Weekday Price Difference
weekend_prices = df[df['period'] == 'weekends']['realSum']
weekday_prices = df[df['period'] == 'weekdays']['realSum']

t_stat, p_value = stats.ttest_ind(weekend_prices, weekday_prices)
print(f"=== Weekend vs Weekday Price Comparison ===")
print(f"T-statistic: {t_stat:.4f}")
print(f"P-value: {p_value:.4f}")
print(f"Significant difference: {'Yes' if p_value < 0.05 else 'No'}")
print(f"Weekend average: {weekend_prices.mean():.2f}")
print(f"Weekday average: {weekday_prices.mean():.2f}")


In [ ]:
# Statistical Test 2: Superhost vs Regular Host Price Comparison
superhost_prices = df[df['host_is_superhost'] == True]['realSum']
regular_prices = df[df['host_is_superhost'] == False]['realSum']

t_stat, p_value = stats.ttest_ind(superhost_prices, regular_prices)
print(f"=== Superhost vs Regular Host Price Comparison ===")
print(f"T-statistic: {t_stat:.4f}")
print(f"P-value: {p_value:.4f}")
print(f"Significant difference: {'Yes' if p_value < 0.05 else 'No'}")
print(f"Superhost average: {superhost_prices.mean():.2f}")
print(f"Regular host average: {regular_prices.mean():.2f}")


In [ ]:
# Statistical Test 3: Correlation between price and satisfaction
correlation, p_value = stats.pearsonr(df['realSum'], df['guest_satisfaction_overall'])
print(f"=== Price vs Guest Satisfaction Correlation ===")
print(f"Correlation coefficient: {correlation:.4f}")
print(f"P-value: {p_value:.4f}")
print(f"Significant correlation: {'Yes' if p_value < 0.05 else 'No'}")


## 6. Feature Engineering & Advanced Analysis


In [ ]:
# Feature Engineering: Price segments
def categorize_price(price):
    if price < df['realSum'].quantile(0.33):
        return 'Budget'
    elif price < df['realSum'].quantile(0.67):
        return 'Mid-range'
    else:
        return 'Premium'

df['price_segment'] = df['realSum'].apply(categorize_price)

# Feature Engineering: Location quality
df['location_quality'] = pd.cut(df['location_score'], 
                                bins=[0, 30, 50, 100], 
                                labels=['Low', 'Medium', 'High'])

print("=== Price Segment Distribution ===")
print(df['price_segment'].value_counts())
print("\n=== Location Quality Distribution ===")
print(df['location_quality'].value_counts())


In [ ]:
# Advanced Analysis: Market positioning by city
market_analysis = df.groupby(['city', 'price_segment']).agg({
    'realSum': ['count', 'mean'],
    'guest_satisfaction_overall': 'mean'
}).round(2)

market_analysis.columns = ['listing_count', 'avg_price', 'avg_satisfaction']
market_analysis = market_analysis.reset_index()

print("=== Market Positioning by City and Price Segment ===")
print(market_analysis.sort_values(['city', 'price_segment']))


In [ ]:
# Advanced Analysis: Value proposition analysis (price per person vs satisfaction)
value_analysis = df.groupby('city').agg({
    'price_per_person': 'mean',
    'guest_satisfaction_overall': 'mean',
    'realSum': 'count'
}).round(2)

value_analysis.columns = ['avg_price_per_person', 'avg_satisfaction', 'listing_count']
value_analysis['value_score'] = value_analysis['avg_satisfaction'] / value_analysis['avg_price_per_person']
value_analysis = value_analysis.sort_values('value_score', ascending=False)

print("=== Value Proposition Analysis (Satisfaction per Price Unit) ===")
print(value_analysis)


## 7. Business Intelligence Insights & Recommendations


In [ ]:
# Key Insights Summary
print("=" * 60)
print("BUSINESS INTELLIGENCE INSIGHTS SUMMARY")
print("=" * 60)

# Insight 1: Most expensive and cheapest cities
city_prices = df.groupby('city')['realSum'].mean().sort_values(ascending=False)
print(f"\n1. PRICING ANALYSIS:")
print(f"   Most expensive city: {city_prices.index[0]} (€{city_prices.iloc[0]:.2f})")
print(f"   Most affordable city: {city_prices.index[-1]} (€{city_prices.iloc[-1]:.2f})")
print(f"   Price range: €{city_prices.max() - city_prices.min():.2f}")

# Insight 2: Supply concentration
city_supply = df.groupby('city').size().sort_values(ascending=False)
print(f"\n2. SUPPLY ANALYSIS:")
print(f"   City with most listings: {city_supply.index[0]} ({city_supply.iloc[0]} listings)")
print(f"   City with fewest listings: {city_supply.index[-1]} ({city_supply.iloc[-1]} listings)")
print(f"   Supply concentration: Top 3 cities have {city_supply.head(3).sum()} listings ({city_supply.head(3).sum()/len(df)*100:.1f}%)")

# Insight 3: Room type distribution
room_dist = df['room_type'].value_counts(normalize=True) * 100
print(f"\n3. ROOM TYPE DISTRIBUTION:")
for room_type, pct in room_dist.items():
    print(f"   {room_type}: {pct:.1f}%")

# Insight 4: Superhost impact
superhost_impact = df.groupby('host_is_superhost').agg({
    'realSum': 'mean',
    'guest_satisfaction_overall': 'mean'
})
print(f"\n4. SUPERHOST IMPACT:")
print(f"   Superhost average price: €{superhost_impact.loc[True, 'realSum']:.2f}")
print(f"   Regular host average price: €{superhost_impact.loc[False, 'realSum']:.2f}")
print(f"   Superhost average satisfaction: {superhost_impact.loc[True, 'guest_satisfaction_overall']:.1f}")
print(f"   Regular host average satisfaction: {superhost_impact.loc[False, 'guest_satisfaction_overall']:.1f}")

# Insight 5: Weekend premium
weekend_price = df[df['period'] == 'weekends']['realSum'].mean()
weekday_price = df[df['period'] == 'weekdays']['realSum'].mean()
premium_pct = ((weekend_price - weekday_price) / weekday_price) * 100
print(f"\n5. WEEKEND PRICING:")
print(f"   Weekend average: €{weekend_price:.2f}")
print(f"   Weekday average: €{weekday_price:.2f}")
print(f"   Weekend premium: {premium_pct:.1f}%")

# Insight 6: Best value cities
print(f"\n6. BEST VALUE CITIES (High Satisfaction, Low Price):")
for idx, (city, row) in enumerate(value_analysis.head(3).iterrows(), 1):
    print(f"   {idx}. {city}: Satisfaction {row['avg_satisfaction']:.1f}, Price/person €{row['avg_price_per_person']:.2f}")

print("\n" + "=" * 60)


In [ ]:
# Business Recommendations
print("=" * 60)
print("BUSINESS RECOMMENDATIONS")
print("=" * 60)

recommendations = [
    "1. MARKET EXPANSION OPPORTUNITIES:",
    f"   - Consider expanding supply in {city_supply.index[-1]} (lowest supply)",
    f"   - Focus on {city_prices.index[-1]} for budget-friendly listings",
    f"   - Premium market opportunities in {city_prices.index[0]}",
    "",
    "2. PRICING STRATEGY:",
    f"   - Implement dynamic pricing: {premium_pct:.1f}% premium for weekends",
    "   - Superhosts can command higher prices (validate with demand data)",
    "   - Price segmentation shows clear market tiers (Budget/Mid-range/Premium)",
    "",
    "3. QUALITY IMPROVEMENT:",
    "   - Superhost status correlates with higher satisfaction",
    "   - Focus on improving cleanliness ratings (key satisfaction driver)",
    "   - Location quality (attr_index + rest_index) influences pricing",
    "",
    "4. ROOM TYPE OPTIMIZATION:",
    f"   - {room_dist.index[0]} dominates market ({room_dist.iloc[0]:.1f}%)",
    "   - Consider incentivizing entire home/apt listings in underserved areas",
    "",
    "5. DATA COLLECTION RECOMMENDATIONS:",
    "   - Collect booking data to analyze demand patterns",
    "   - Track seasonal trends and special events impact",
    "   - Monitor occupancy rates by city and room type",
    "   - Analyze cancellation rates and booking patterns"
]

for rec in recommendations:
    print(rec)

print("\n" + "=" * 60)


## 8. Export Results for Further Analysis


In [ ]:
# Export key metrics
city_summary = df.groupby('city').agg({
    'realSum': ['mean', 'median', 'std', 'count'],
    'guest_satisfaction_overall': 'mean',
    'host_is_superhost': 'mean',
    'person_capacity': 'mean',
    'bedrooms': 'mean'
}).round(2)

city_summary.columns = ['avg_price', 'median_price', 'price_std', 'listing_count',
                        'avg_satisfaction', 'superhost_pct', 'avg_capacity', 'avg_bedrooms']
city_summary = city_summary.reset_index()

# Save to CSV
city_summary.to_csv('city_summary_statistics.csv', index=False)
print("City summary statistics exported to 'city_summary_statistics.csv'")

# Display summary
print("\n=== City Summary Statistics ===")
print(city_summary)


## 9. Conclusion

This analysis provides a comprehensive view of Airbnb supply patterns across European cities. Key findings include:

- **Pricing Variations**: Significant price differences across cities
- **Supply Concentration**: Uneven distribution of listings across cities
- **Quality Indicators**: Superhost status and location quality impact pricing
- **Market Segmentation**: Clear price segments (Budget/Mid-range/Premium)
- **Weekend Premium**: Weekend pricing shows consistent premium

**Next Steps for Deep Dive Analysis:**
1. Integrate demand/booking data for complete market analysis
2. Time-series analysis with seasonal trends
3. Predictive modeling for pricing optimization
4. Geographic clustering analysis
5. Competitive analysis by city
